First, let's import the required libraries:

In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


Next, let's define some variables to store the dataset path, image size, and number of classes:



In [ ]:
DATADIR = "./skin-lesion-analysis-towards-melanoma-detection/"
IMG_SIZE = 100
NUM_CLASSES = 2


The DATADIR variable should be replaced with the path to the dataset directory on your machine. The IMG_SIZE variable defines the size of the images after resizing. The NUM_CLASSES variable defines the number of classes in the dataset.

Now, let's define a function to preprocess the images:

In [ ]:
def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the image to IMG_SIZE x IMG_SIZE
    resized = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))
    # Normalize the image to have pixel values between 0 and 1
    normalized = resized / 255.0
    # Reshape the image to have a single color channel
    reshaped = np.reshape(normalized, (IMG_SIZE, IMG_SIZE, 1))
    return reshaped


This function takes an image as input, converts it to grayscale, resizes it to IMG_SIZE x IMG_SIZE, normalizes it to have pixel values between 0 and 1, and reshapes it to have a single color channel.

Now, let's load the images and labels from the dataset directory:

In [ ]:
images = []
labels = []

for label in range(NUM_CLASSES):
    path = os.path.join(DATADIR, str(label))
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_data = cv2.imread(img_path)
        img_data = preprocess_image(img_data)
        images.append(img_data)
        labels.append(label)

# Convert the images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the images and labels
images, labels = shuffle(images, labels, random_state=1)


This code loops over each class in the dataset directory, loads each image, preprocesses it using the preprocess_image() function, and appends it to the images list along with its label. Finally, it converts the images and labels lists to NumPy arrays and shuffles them.

Now, let's split the dataset into training and testing sets:



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=1)


This code uses the train_test_split() function from the sklearn library to split the images and labels arrays into training and testing sets, with 20% of the data used for testing.

Next, let's define the CNN model:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

This code defines a sequential model with the following layers:

1. `Conv2D` layer with 32 filters, kernel size of 3x3, ReLU activation function, and input shape of `IMG_SIZE x IMG_SIZE x 1`.
2. `MaxPooling2D` layer with pool size of 2x2.
3. `Conv2D` layer with 64 filters, kernel size of 3x3, ReLU activation function.
4. `MaxPooling2D` layer with pool size of 2x2.
5. `Conv2D` layer with 128 filters, kernel size of 3x3, ReLU activation function.
6. `MaxPooling2D` layer with pool size of 2x2.
7. `Flatten` layer to convert the output of the convolutional layers to a 1D array.
8. `Dense` layer with 128 neurons and ReLU activation function.
9. `Dense` layer with `NUM_CLASSES` neurons and softmax activation function.

Finally, let's compile and train the model:

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

This code compiles the model using the Adam optimizer, sparse categorical crossentropy loss function, and accuracy metric. It then trains the model for 10 epochs on the training data, with validation on the testing data.

That's it! This program should be able to classify skin cancer images and non-cancer images using a CNN.